In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

ds = fetch_openml('mnist_784', as_frame=False)

x_train, x_test, y_train, y_test = train_test_split(ds.data, ds.target,test_size=0.2, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
a = x_train[0].reshape((28,28))
plt.imshow(a)

In [ ]:
# down-sampling
import cv2
b = cv2.resize(b, (14,14))
plt.imshow(b)